# Segundo modelo de regressão logística

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-muted")
%matplotlib inline

# Lendo os dados de treino e teste

---

In [72]:
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

# Explorando os dados 

---

In [73]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


---

## Dicionário das variáveis

Podemos observar que temos as seguinte estrutura de variáveis no banco:

In [74]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


---

O que significa cada uma, segundo o [Kaggle](https://www.kaggle.com/c/titanic/data) é:

1. **PassengerId**: Um ID para identificar cada passageiro;
2. **Survived**: Se ele sobreviveu (0 = Não, 1 = Sim) - Qualitativa;
3. **Pclass**: Ticket que indica qual classe o passageiro estava (1 = 1st, 2 = 2nd, 3 = 3rd) - Qualitativa;
4. **Name**: Nome - Qualitativa;
5. **Sex**: Sexo - Qualitativa;
6. **Age**: Idade - Quantitativa;
7. **SibSp**: Nº de irmãos / cônjuges a bordo do Titanic - Quantitativa;
8. **Parch**: Nº de pais / filhos a bordo do Titanic - Quantitativa;
9. **Ticket**: Nº do ticket - Qualitativa;
10. **Fare**: Tarifa do passageiro - Quantitativa;
11. **Cabin**: Número da cabine - Qualitativa;
12. **Embarked**: Porto em que o passageiro embarcou - Qualitativa.

# Pré-processamento dos dados

---



## Concatenar os dados de treino e teste

In [75]:
survived = treino["Survived"]

survived

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [76]:
treino.drop(["Survived"], axis = 1, inplace = True)

treino_index = treino.shape[0]
teste_index = teste.shape[0]

banco_geral = pd.concat(objs = [treino, teste], axis = 0).reset_index(drop = True)

In [77]:
banco_geral.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [78]:
banco_geral.shape

(1309, 11)

## Selecionando as variáveis

In [79]:
banco_geral = banco_geral.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

In [80]:
banco_geral.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


## Transformação de variáveis

1. *Age*: A idade será substituída por uma variável chamada Faixa Etária e os valores nulos terão uma classe SI.
2. *Pclass*: Recodificar ela para variável qualitativa.
3. *Fare*: Devido a distribuição da variável Fare possuir outliers, iremos utilizar o log.
4. *Embarked*: Aqui podemos criar uma categoria a mais também dizendo que não sabemos de qual porto a pessoa embarcou.
5. *SibSp + Parch*: Criamos uma variável chamada "Qtd_familiares", que diz o total de parentes que cada passageiro.
6. *Onehotencode* nas variáveis qualitativas

### Age

In [82]:
def mudar_idade(idade):
    
    if idade >= 0 and idade <= 20:
        return("0-20")
    elif idade >= 21 and idade <= 40:
        return("21-40")
    elif idade >= 41 and idade <= 60:
        return("41-60")
    elif idade >= 61 and idade <= 80:
        return("61-80")
    else: 
        return("SI")
        
banco_geral["Age"] = banco_geral["Age"].apply(mudar_idade)

In [83]:
banco_geral["Age"]

0       21-40
1       21-40
2       21-40
3       21-40
4       21-40
        ...  
1304       SI
1305    21-40
1306    21-40
1307       SI
1308       SI
Name: Age, Length: 1309, dtype: object

# Parei aqui (próximo é o Pclcass)

## Pclass

In [11]:
banco_geral["Pclass"] = banco_geral["Pclass"].astype(str)

### Embarked

In [13]:
banco_geral["Embarked"] = banco_geral["Embarked"].fillna("SI")

### Fare

Substituir pela moda também.

In [14]:
banco_geral["Fare"] = banco_geral["Fare"].fillna(banco_geral["Fare"].mode()[0])

banco_geral.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

## OneHotEncode nas variáveis qualitativas/categóricas

In [15]:
banco_geral = pd.get_dummies(banco_geral)

In [16]:
banco_geral.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_SI
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1,0
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1,0
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1,0
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1,0


## Separando em treino e teste novamente

In [17]:
treino1 = banco_geral.iloc[:treino_index]

print("Dimensões do novos dados de treino")

treino1.shape

Dimensões do novos dados de treino


(891, 13)

In [18]:
teste1 = banco_geral.iloc[:teste_index]

print("Dimensões do novos dados de teste")

teste1.shape

Dimensões do novos dados de teste


(418, 13)

In [19]:
treino1 = treino1.assign(Survived = survived)

treino1.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_SI,Survived
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1,0,0
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0,0,1
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1,0,1
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1,0,1
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1,0,0


## Separando os dados de treino em validação também

In [20]:
from sklearn.model_selection import train_test_split

x_treino, x_valid, y_treino, y_valid = train_test_split(treino1.drop("Survived", axis = 1), treino1["Survived"], train_size = 0.5, random_state = 1234)

In [71]:
print("Os dados de treino possui dimensões:", treino1.shape)
print("---")
print("x_treino possui dimensões:", x_treino.shape)
print("---")
print("y_treino possui dimensões:", y_treino.shape)
print("---")
print("x_valid possui dimensões:", x_valid.shape)
print("---")
print("y_valid possui dimensões:", y_valid.shape)

Os dados de treino possui dimensões: (891, 14)
---
x_treino possui dimensões: (445, 13)
---
y_treino possui dimensões: (445,)
---
x_valid possui dimensões: (446, 13)
---
y_valid possui dimensões: (446,)


# Regressão logística

---




In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(x_treino, y_treino)

C:\Users\Rafael\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
previsao = logreg.predict(x_valid)

previsao[:6]

array([1, 0, 0, 1, 0, 0], dtype=int64)

## Matriz de confusão

In [74]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_valid, previsao)

print(confusion_matrix)

[[239  43]
 [ 43 121]]
